Required Modules import

In [ ]:
import os
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from keras.utils.vis_utils import model_to_dot


print(tf.__version__)
print(np.__version__)

2.4.0
1.19.4


Loading Images

In [ ]:
labelNames = ['cardigan_1_5',
              'coat_1_10','coat_1_11','coat_1_12',
              'collar_0_3','collar_0_4',
              'dress_2_0','dress_2_1','dress_2_2',
              'hood_0_6',
              'jacket_1_1','jacket_1_2','jacket_1_3','jacket_1_4','jacket_1_6','jacket_1_7','jacket_1_8','jacket_1_9','jacket_1_17',
              'jsuit_2_3',
              'lpadding_1_13',
              'lpants_3_0','lpants_3_1','lpants_3_2','lpants_3_3','lpants_3_5','lpants_3_6',
              'lsleeve_0_1','lsleeve_0_5','lsleeve_0_7','lsleeve_0_8','lsleeve_1_0','lsleeve_1_18',
              'nsleeve_0_2',
              'skirt_4_0','skirt_4_1','skirt_4_2',
              'spadding_1_14',
              'spants_3_4',
              'ssleeve_0_0',
              'vest_1_15','vest_1_16'
             ]
print(len(labelNames))
classDic = {0:2,
            1:2, 2:2, 3:2,
            4:2, 5:1,
            6:5, 7:5, 8:5,
            9:2,
            10:2, 11:2, 12:2, 13:2, 14:2, 15:2, 16:2, 17:2, 18:2, 
            19:5,
            20:2,
            21:3, 22:3, 23:3, 24:3, 25:3, 26:3, 
            27:2, 28:2, 29:2, 30:2, 31:2, 32:2, 
            33:0,
            34:4, 35:4, 36:4,
            37:2,
            38:3,
            39:1,
            40:0, 41:0
           }
print(len(classDic))
classNames = ['nsleeve', 'ssleeve', 'lsleeve', 'pants', 'skirt', 'dress']

dir = "/Users/siyeon/Desktop/dataproject/data/collected/image"

42
42


Create numpy data


In [ ]:
import numpy as np
import pandas as pd

x_train = []
y_train = []
x_test = []
y_test = []
fileNames = []

for idx, cat in enumerate(labelNames):

    label = [0 for i in range(len(classNames))]
    label[classDic[idx]] = 1

    imgDir = dir + "/" + cat
    files = glob.glob(imgDir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    
    i = 0
    lst = []
    for i in range(len(files)) :
        lst.append(i)
    lst = pd.DataFrame(lst, columns = ['idx'])

    trainSet = lst.apply(sampling, per=0.95)
    trainSet = trainSet.sort_index()

    testSet = lst.drop(lst.index[trainSet.index])
    testSet = testSet.sort_index()

    trainSet = trainSet['idx'].values.tolist()
    testSet = testSet['idx'].values.tolist()
    
    for f in range(len(trainSet)):
        img = Image.open(files[trainSet[f]])
        img = img.convert("RGB")
        data = np.asarray(img)
        
        if f == 0:
            print(label)
            
        x_train.append(data)
        y_train.append(label)
    
    for f in range(len(testSet)):
        img = Image.open(files[testSet[f]])
        img = img.convert("RGB")
        data = np.asarray(img)
        
        if f == 0:
            print(label)
            
        x_test.append(data)
        y_test.append(label)
        fileNames.append(files[testSet[f]])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_train.shape)
print(y_train.shape)

print("ok", len(x_train), len(x_test))

In [ ]:
import os, glob, numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

import tensorflow as tf
config = tf.config.experimental

imageLoad = np.load("/content/drive/MyDrive/Colab Notebooks/imageDataRaw.npz")
x_train = imageLoad['x_train']
y_train = imageLoad['y_train']
x_test = imageLoad['x_test']
y_test = imageLoad['y_test']
print(x_train.shape)

(16599, 125, 125, 3)


In [ ]:
x_train = x_train.astype(float) / 255
x_test = x_test.astype(float) / 255

Modeling


In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

base_model = InceptionResNetV2(include_top=False, pooling='avg')
outputs = Dense(len(classNames), activation='softmax')(base_model.output)

In [ ]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  781


In [ ]:
model = Model(base_model.inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'
    
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
checkpointer = ModelCheckpoint(filepath='/content/drive/Shareddrives/TimmyRoom/check',
                              verbose=1, save_best_only=True)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.1)
print(x_train.shape)
print(x_valid.shape)

Training


In [ ]:
history = model.fit(x_train, y_train, batch_size=32,steps_per_epoch=x_train.shape[0]//32,
                    epochs = 100, verbose=2, callbacks=[checkpointer, early_stopping],
                    validation_data=(x_valid, y_valid))

In [ ]:
from keras.models import load_model
model.save('/content/drive/MyDrive/Colab Notebooks/aug_all.weights.best.hdf5')

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/aug_all.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

print('\n','Test accuracy:', score[1])

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()